In [33]:
import random
random.seed(38)
arr = np.array([round(random.uniform(-5, 5), 5) for i in range(175)])
print(arr)

[ 1.39474 -0.70979  2.24299 -4.33769  1.97863 -1.25068  1.08036 -3.29909
 -1.55433 -2.33037 -1.70521  2.26597 -0.26949  4.62792  0.82841 -2.76707
  1.12798  0.28049 -3.59171  0.26846 -4.88318 -1.25593  0.36387  1.86223
  4.59414  1.14663 -4.75422 -0.24466 -2.49799  2.47286  3.93631 -4.1322
  2.7163  -2.0678  -3.1717  -0.46467 -0.81243 -2.12473 -2.36136 -0.66502
  1.89474 -3.36071 -1.27174 -2.60801  2.28585 -2.51953  2.39677 -4.71227
  3.55744 -0.72273  4.34169  2.96006  3.42565 -4.69841  2.48449  0.25369
  3.86223  0.66685 -1.52719  4.04277  4.94745 -2.79086 -3.542   -0.58329
  1.5757  -3.38586  0.85705  0.42934  2.99043 -3.54643 -3.59494  0.18462
  2.52281  0.86928  2.4307  -2.36297 -4.67203  1.89295  1.47189 -4.72687
 -4.14123  1.98443  1.66624 -2.91671 -2.38558 -0.81699 -4.79314 -1.99932
 -2.60104 -4.58021  0.24658  4.41245 -4.22231  1.9273   1.14141 -2.14774
  1.4348   2.73187 -2.39972 -0.90728  1.2709  -0.59845 -3.14333 -0.96776
  2.10727  0.85762 -1.76021 -2.19186 -3.30931  1.667

In [34]:
import numpy as np
from multiprocessing import Process, Queue, cpu_count
from queue import Empty
import numba

def precompute_next_j(a):
    """Precompute next valid indices."""
    n = len(a)
    return [np.where(a[i+1:] >= a[i])[0] + (i + 1) for i in range(n)]

@numba.jit(nopython=True)
def generate_subsequences(start_i, k, all_j, offsets):
    """Generate subsequences using Numba for speed."""
    results = []
    stack = [(start_i, 0, [start_i])]  # (last, j_idx, path)
    while stack:
        last, j_idx, path = stack.pop()
        if len(path) == k:
            results.append(path)
            continue
        next_j_start = offsets[last]
        next_j_end = offsets[last + 1]
        for idx in range(j_idx, next_j_end - next_j_start):
            j = all_j[next_j_start + idx]
            stack.append((j, 0, path + [j]))
    return results

def worker(start_indices, k, all_j, offsets, queue):
    """Worker sends subsequences one at a time."""
    for start_i in start_indices:
        subsequences = generate_subsequences(start_i, k, all_j, offsets)
        for seq in subsequences:
            queue.put(seq)
    queue.put(None)

def non_decreasing_subsequences_iterator(a, k, num_processes=cpu_count()):
    """Iterator with Numba optimization."""
    next_j_list = precompute_next_j(a)
    all_j = np.concatenate(next_j_list)
    offsets = np.cumsum([0] + [len(j) for j in next_j_list])
    queue = Queue(maxsize=1000)  # Smaller queue
    processes = []
    start_indices_list = np.array_split(range(len(a)), num_processes)

    for start_indices in start_indices_list:
        p = Process(target=worker, args=(start_indices, k, all_j, offsets, queue))
        p.start()
        processes.append(p)

    finished_processes = 0
    while finished_processes < num_processes:
        try:
            item = queue.get(timeout=1)
            if item is None:
                finished_processes += 1
            else:
                yield item
        except Empty:
            continue

    for p in processes:
        p.join()

In [35]:
import bisect

def lnds(nums):
    tails = []
    for num in nums:
        idx = bisect.bisect_right(tails, num)
        if idx == len(tails):
            tails.append(num)
        else:
            tails[idx] = num
    return len(tails)

k = lnds(arr)
print(k)

22


In [36]:
def float_value_iterator(array, k):
    base_iterator = non_decreasing_subsequences_iterator(array, k)
    for indices in base_iterator:
        yield [array[i] for i in indices]

In [37]:
import time
iterator = iter(float_value_iterator(arr, k))
start = time.time()
while True:
  try:
    print(next(iterator))
  except StopIteration: break
stop = time.time()
print(stop - start)

[np.float64(-4.33769), np.float64(-3.29909), np.float64(-2.76707), np.float64(-2.49799), np.float64(-2.36136), np.float64(-1.27174), np.float64(-0.72273), np.float64(0.25369), np.float64(0.66685), np.float64(0.85705), np.float64(0.86928), np.float64(1.47189), np.float64(1.66624), np.float64(1.9273), np.float64(2.10727), np.float64(2.78729), np.float64(3.66156), np.float64(4.01784), np.float64(4.21443), np.float64(4.67615), np.float64(4.74933), np.float64(4.97219)]
[np.float64(-4.33769), np.float64(-3.29909), np.float64(-2.76707), np.float64(-2.49799), np.float64(-2.36136), np.float64(-1.27174), np.float64(-0.72273), np.float64(0.25369), np.float64(0.66685), np.float64(0.85705), np.float64(0.86928), np.float64(1.47189), np.float64(1.66624), np.float64(1.9273), np.float64(2.73187), np.float64(2.78729), np.float64(3.66156), np.float64(4.01784), np.float64(4.21443), np.float64(4.67615), np.float64(4.74933), np.float64(4.97219)]
[np.float64(-4.33769), np.float64(-3.29909), np.float64(-2.767